In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

In [2]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.11.0


In [3]:
'''
加载并准备mnist数据集。图像的像素值范围从 0 到 255。通过将值除255以将这些值缩放到 0 到 1 的范围255.0。
这还将样本数据从整数转换为浮点数
'''

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 1s 0us/step


In [4]:
#建立一个机器学习模型：
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
'''
Sequential 对于堆叠层很有用，其中每一层都有一个输入张量和一个输出张量。层是具有已知数学结构的函数，可以重复使用并具有可训练的变量。
大多数 TensorFlow 模型由层组成。此模型使用flatten, dense和dropout层。
'''

In [5]:
#对于每个示例，模型返回一个logits或log-odds分数向量，每个类一个。
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.37804273, -0.4494687 , -0.14929909,  0.47527167,  0.87909317,
        -0.17083329,  0.12572944,  0.4083663 , -0.4640041 ,  0.61762613]],
      dtype=float32)

In [6]:
# 该tf.nn.softmax函数将这些 logits 转换为每个类别的概率：
tf.nn.softmax(predictions).numpy()

array([[0.05631997, 0.05243756, 0.07079531, 0.1322063 , 0.19798377,
        0.06928708, 0.09320686, 0.12365039, 0.05168087, 0.1524319 ]],
      dtype=float32)

In [7]:
#使用 定义训练损失函数losses.SparseCategoricalCrossentropy：
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
'''
损失函数采用地面真值向量和逻辑向量，并返回每个示例的标量损失。此损失等于真实类别的负对数概率：如果模型确定类别正确，则损失为零。

这个未经训练的模型给出的概率接近随机（每个类别为 1/10），因此初始损失应该接近-tf.math.log(1/10) ~= 2.3.'''

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.6694968

In [9]:
'''
在开始训练之前，使用 Keras 配置和编译模型Model.compile。
将类设置optimizer为adam，将 设置loss为您之前定义的函数，并通过将参数设置为 来loss_fn指定要为模型评估的指标。metricsaccuracy'''
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [10]:
#使用该Model.fit方法调整您的模型参数并最大程度地减少损失：
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2966 - accuracy: 0.9144
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1448 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1085 - accuracy: 0.9668
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0897 - accuracy: 0.9724
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0752 - accuracy: 0.9771


In [11]:
#该方法通常在验证集或测试集Model.evaluate上检查模型的性能。
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0740 - accuracy: 0.9776 - 754ms/epoch - 2ms/step


[0.07400603592395782, 0.9775999784469604]

In [ ]:
#现在，图像分类器在此数据集上的训练准确度约为 98%。要了解更多信息，请阅读TensorFlow 教程。
#如果你想让你的模型返回一个概率，你可以包装训练好的模型，并将 softmax 附加到它：
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])
probability_model(x_test[:5])